In [1]:
!ls

date.py					 Python3数据分析与挖掘建模实战
DM_Airbnb				 Python_cookbook
environment.yml				 README.md
Hands-on_ML_with_sklearn_and_tensorflow  Sklearn
HR.csv					 untitled
img					 Untitled1.ipynb
knn_clf					 Untitled2.ipynb
Matplotlib				 Untitled.ipynb
MLDemo.ipynb				 未命名


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
#sl:satisfaction_level--False:MinMaxScaler,True:StandardScaler
#le:last_evaluation--False:MinMaxScaler,True:StandardScaler
#npr:number_project--False:MinMaxScaler,True:StandardScaler
#amh:average_montly_hours--
#tsc:time_spend_company--
#wa:Work_accident--
#pl5:promotion_last_5years--

#dp:department--False:LabelEncoding,True:OneHotEncoding
#slr:salary--
def hr_preprocessing(sl=False,le=False,npr=False,
                     amh=False,tsc=False,wa=False,pl5=False,dp=False,slr=False,
                     lower_d=False,ld_n=1):
    df=pd.read_csv('./HR.csv')
    
    #2. 清洗数据
    df=df.dropna(subset=['satisfaction_level','last_evaluation'])
    df=df[df['satisfaction_level']<=1][df['salary']!='nme']
    df=df.rename(columns={'sales':'department'})
    #1. 标注
    label=df["left"]
    df=df.drop("left",axis=1)
    #3. 特征选择
    #4. 特征处理
    scaler_lst=[sl,le,npr,amh,tsc,wa,pl5]
    column_lst=['satisfaction_level','last_evaluation','number_project',
                'average_montly_hours','time_spend_company','Work_accident','promotion_last_5years']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:
            df[column_lst[i]]=StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
            
    scaler_lst=[slr,dp]
    column_lst=['salary','department']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i]=="salary":
                df[column_lst[i]]=[map_salary(s) for s in df["salary"].values]
            else:
                df[column_lst[i]]=LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:
            df=pd.get_dummies(df,columns=[column_lst[i]])
    
    if lower_d:
        #return LinearDiscriminantAnalysis(n_components=ld_n)
        return PCA(n_components=ld_n).fit_transform(df.values),label
    return df,label
d=dict([("low",0),('medium',1),('high',2)])
def map_salary(s):
    return d.get(s,0)

In [3]:
def hr_modeling(features,label):
    from sklearn.model_selection import train_test_split
    f_v=features.values
    l_v=label.values
    X_tt,X_validation,Y_tt,Y_validation=train_test_split(f_v,l_v,test_size=0.2)
    X_train,X_test,Y_train,Y_test=train_test_split(X_tt,Y_tt,test_size=0.25)
    print(len(X_train),len(X_validation),len(X_test))
    
    #KNN
    from sklearn.neighbors import NearestNeighbors,KNeighborsClassifier
    knn_clf=KNeighborsClassifier(n_neighbors=3)
    knn_clf_n5=KNeighborsClassifier(n_neighbors=5)
    knn_clf.fit(X_train,Y_train)
    knn_clf_n5.fit(X_train,Y_train)
    Y_pred=knn_clf.predict(X_validation)
    Y_pred_n5=knn_clf_n5.predict(X_validation)
    from sklearn.metrics import accuracy_score,recall_score,f1_score
    print("验证集")
    print("ACC:",accuracy_score(Y_validation,Y_pred))
    print("REC:",recall_score(Y_validation,Y_pred))
    print("F-Score:",f1_score(Y_validation,Y_pred))
    print("--n=5---")
    print("ACC:",accuracy_score(Y_validation,Y_pred_n5))
    print("REC:",recall_score(Y_validation,Y_pred_n5))
    print("F-Score:",f1_score(Y_validation,Y_pred_n5))
    
    #用测试集衡量
    Y_pred_test=knn_clf.predict(X_test)
    print("测试得分")
    print("ACC:",accuracy_score(Y_test,Y_pred_test))
    print("REC:",recall_score(Y_test,Y_pred_test))
    print("F-Score:",f1_score(Y_test,Y_pred_test))
    
    #模型存储
    from sklearn.externals import joblib
    joblib.dump(knn_clf,"knn_clf") #保存
    #knn_clf=joblib.load("knn_clf") #使用时再次加载

In [3]:
def main():
    features,label=hr_preprocessing()
    hr_modeling(features,label)
if __name__=='__main__':
    main()

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.

8999 3000 3000
验证集
ACC: 0.9493333333333334
REC: 0.9088397790055248
F-Score: 0.896457765667575
--n=5---
ACC: 0.953
REC: 0.9060773480662984
F-Score: 0.9029593943565039
测试得分
ACC: 0.9516666666666667
REC: 0.9118065433854907
F-Score: 0.8983882270497547


In [16]:
features,label=hr_preprocessing()

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.

In [17]:
features.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,department,salary
0,0.318681,0.265625,0.0,0.285047,0.125,0.0,0.0,0.777778,0.0
1,0.780220,0.781250,0.6,0.775701,0.500,0.0,0.0,0.777778,0.5
2,0.021978,0.812500,1.0,0.822430,0.250,0.0,0.0,0.777778,0.5
3,0.692308,0.796875,0.6,0.593458,0.375,0.0,0.0,0.777778,0.0
4,0.307692,0.250000,0.0,0.294393,0.125,0.0,0.0,0.777778,0.0


In [18]:
f_v=features.values

In [19]:
l_v=label.values

## 神经网络

In [8]:
import tensorflow as tf

In [11]:
tf.__version__

'1.12.0'

In [10]:
from tensorflow.keras.models import Sequential

In [14]:
from tensorflow.keras.layers import Dense,Activation

In [15]:
from tensorflow.keras.optimizers import SGD

In [34]:
mdl=Sequential()
mdl.add(Dense(50,input_dim=len(f_v[0])))
mdl.add(Activation("sigmoid"))
mdl.add(Dense(2))
mdl.add(Activation("softmax"))

sgd=SGD(lr=0.05)

mdl.compile(lose="mean_squared_error",optimizer='adam')

#mdl.fit(X_train,np.array([[0,1] if i ==1 else [1,0] for i in y_train]),nb_epoch=1000,batch_size=2048)

ValueError: The model cannot be compiled because it has no loss to optimize.

GBDT

In [35]:
from sklearn.ensemble import GradientBoostingClassifier

In [36]:
gbdt=GradientBoostingClassifier(max_depth=6,n_estimators=100)